<a href="https://colab.research.google.com/github/Shankar-Pankhania/Twitter_Sentiment_Analysis_on_Bitcoin/blob/main/count_sentiments_per_hour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Dissertation/

In [6]:
import numpy as np

In [ ]:
import pandas as pd

df = pd.read_csv('bitcoin_tweets.csv')

Download Botometer

In [ ]:
pip install botometer

Detect bots

In [ ]:
from botometer import Botometer

#samples 1000 tweets from the dataset
sample_entire_dataset = df.sample(n=1000)

rapid_api_key = '0dd6bfa718mshf0f360df07771b5p13cce4jsnae7a91cae654'

#Get your Twitter API credentials and enter them here
twitter_tokens = {
  'consumer_key' : 'ngUiFH7BGoDjoeRYFp6J4CISz',
  'consumer_secret' : 'sLGTulZAxLThdluHb7RyGTF08jpc75NowJkwj8DaNNG8P7zbeA',
  'access_key' : '2387367300-CsyGg6vQeV7reiGsOzFoRToHi7Jdskyie4Rf3GO',
  'access_secret' : 'J6ivsjOUFilDm7llUgvXe87Cwk8NEGZQyCS9wmgOanINU'
}

botometer = Botometer(mashape_key=rapid_api_key, **twitter_tokens, wait_on_ratelimit=True)

total_bots = []
updateUsernames = sample_entire_dataset['Username']

for i in updateUsernames:
  #validation for the users account
  try:
    #gets the botometer scores of the username
     usernames = botometer.check_account('@'+i)

  except:
    pass
 
  #checks to see if the username is a bot by comparing the overall score against 0.8
  if (usernames['raw_scores']['english']['overall'] >= 0.8):
    total_bots.append(i)

#set sentiment of bot tweet to NULL
for user in total_bots:
  df.loc[df['Username'] == user, ['sentiment']] = np.nan

In [16]:
#The sentiment with Nan represent bots
df

,Unnamed: 0,Username,User ID,Timestamp,Text,Retweet,Likes,processed_tweet,scores,compound,sentiment
0,0,karldickey,1.370000e+18,21/03/2021 00:00,"b""CoinDesk: Will Bitcoin Be the Currency of th...",0,5395,coindesk bitcoin currency world scaramucci qui...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0
1,1,BitcoinLifeClub,1.370000e+18,21/03/2021 00:00,b'Bitcoin Life Club strive to help as many as ...,0,2,bitcoin life club strive help many become owne...,"{'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'comp...",0.4019,1.0
2,2,valueofcrypto,1.370000e+18,21/03/2021 00:00,b'#Bitcoin\nCurrent Price:\n$ 58126.19\n\xe2\x...,0,65,bitcoin ncurrent price xe xac ncryptocurrencie...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NaN
3,3,susiemills__,1.370000e+18,20/03/2021 23:59,b'@MarekZee You can purchase bitcoin at Coinma...,0,42,purchase bitcoin coinmama paybis xe glad show ...,"{'neg': 0.0, 'neu': 0.581, 'pos': 0.419, 'comp...",0.8020,1.0
4,4,CryptoWatchBot,1.370000e+18,20/03/2021 23:59,b'#Investing 74.7% in this optimal #crypto por...,0,98,investing optimal crypto portfolio cash past h...,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.3612,1.0
...,...,...,...,...,...,...,...,...,...,...,...
118590,118590,russeree,1.380000e+18,25/03/2021 23:37,b'#Bitcoin is more money than the dollars I us...,0,69,bitcoin money dollars use today,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0
118591,118591,Sithole_D1,1.380000e+18,25/03/2021 23:37,b'@eNCA I basically thought @Ericwalker01 was...,0,1325,basically thought scam like rest bitcoin accou...,"{'neg': 0.228, 'neu': 0.617, 'pos': 0.154, 'co...",-0.2960,NaN
118592,118592,SweetBabyGus,1.380000e+18,25/03/2021 23:37,"b""Soooo... How are everyone's bags doing? #Bit...",0,10298,soo everyone bags bitcoin cryptocurrencies,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0
118593,118593,Zanele371,1.380000e+18,25/03/2021 23:37,"b""@TheAtlantic @KatherineJWu For all those who...",0,3433,new working home bitcoin trading options nhere...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NaN


Count the sentiments for each hour

In [17]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

count_sentiments_per_hour = df.groupby([pd.Grouper(key='Timestamp',freq='H'), df['sentiment']]).size().reset_index(name='count')

#this will convert rows as column headers and will be the new dataframe
df2 = count_sentiments_per_hour.pivot_table('count', ['Timestamp'], 'sentiment')
df2.rename(columns={-1 : 'count_negative', 0 : 'count_neutral', 1 : 'count_positive'}, inplace=True)

Count average compound each hour

In [18]:
count_compound_per_hour = df.groupby(pd.Grouper(key='Timestamp',freq='H'))['compound'].mean()
df2['compound_total'] = count_compound_per_hour.apply(lambda x: x)

Count average negative sentiment

In [19]:
filter_negative = df[df['sentiment'] == -1]
calculate_mean_negative_hourly = filter_negative.groupby(pd.Grouper(key='Timestamp',freq='H'))['compound'].mean()
df2['sent_negative'] = calculate_mean_negative_hourly.apply(lambda x: x)


Count average positive sentiment

In [ ]:
filter_positive = df[df['sentiment'] == 1]
calculate_mean_positive_hourly = filter_positive.groupby(pd.Grouper(key='Timestamp',freq='H'))['compound'].mean()
df2['sent_positive'] = calculate_mean_positive_hourly.apply(lambda x: x)

Total volume of tweets for each hour

In [25]:
#total volume of tweets include bot tweets
total_volume_of_tweets = df.groupby(pd.Grouper(key='Timestamp',freq='H'))['Text'].count()
df2['total_volume_of_tweets'] = total_volume_of_tweets.apply(lambda x: x)
df2

sentiment,count_negative,count_neutral,count_positive,compound_total,sent_negative,sent_positive,total_volume_of_tweets
Timestamp,,,,,,,
2021-03-20 23:00:00,143,295,367,0.183093,-0.402219,0.484113,997
2021-03-21 00:00:00,108,300,314,0.196218,-0.415027,0.467108,1001
2021-03-21 01:00:00,144,349,355,0.130551,-0.445366,0.491345,997
2021-03-21 02:00:00,149,316,398,0.152785,-0.391138,0.476517,1003
2021-03-21 03:00:00,142,236,439,0.207234,-0.417404,0.509165,1001
...,...,...,...,...,...,...,...
2021-03-25 20:00:00,119,376,393,0.163823,-0.385717,0.522008,1003
2021-03-25 21:00:00,129,332,372,0.147562,-0.429736,0.500219,994
2021-03-25 22:00:00,129,319,360,0.187263,-0.405040,0.530194,1000


Download the dataframe

In [ ]:
from google.colab import files
df2.to_csv('Data_to_hourly_intervals.csv')
files.download('Data_to_hourly_intervals.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>